In [1]:
import pandas as pd
import numpy as np
from numpy import array

In [16]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.optimizers import SGD

In [3]:
inputCSV = '../datasets/prepared/parking-prepared.csv';
data = pd.read_csv(inputCSV)

data['spots_available'] = pd.to_numeric(data['spots_available'])
data['verification_datetime'] = pd.to_datetime(data['verification_datetime'])
data = data.set_index('verification_datetime').asfreq('15min')

data.index = pd.DatetimeIndex(data.index, freq=data.index.inferred_freq)

# Fill All Empty Values with the last 15 min valid value or the medium of the last valid before + the last after value
for i in range(0, len(data)):
    if np.isnan(data.iloc[i]['spots_available']):
        if np.isnan(data.iloc[i+1]['spots_available']):
            data.iloc[i]['spots_available'] = data.iloc[i-1]['spots_available']
        else:
            data.iloc[i]['spots_available'] = (data.iloc[i-1]['spots_available'] + data.iloc[i+1]['spots_available']) / 2

data = data[74:]
print("Total Days: ",len(data)/96)
print(data.head(5)) # First Day Starting at 00:00 with 96 periods
print(data.tail(5)) # Last Day Ending at 23:45 with 96 periods
print(len(data))
data = data.dropna()
print(len(data))

Total Days:  201.0
                       spots_available
verification_datetime                 
2016-10-03 00:00:00             4651.0
2016-10-03 00:15:00             4652.0
2016-10-03 00:30:00             4652.5
2016-10-03 00:45:00             4653.0
2016-10-03 01:00:00             4656.0
                       spots_available
verification_datetime                 
2017-04-21 22:45:00              663.0
2017-04-21 23:00:00              739.0
2017-04-21 23:15:00              773.0
2017-04-21 23:30:00              807.0
2017-04-21 23:45:00              843.0
19296
19296


In [42]:
max_days = 30

newData = data[0:96*max_days]
print("Total Days: ",len(newData)/96)
print(newData.head(5)) # First Day Starting at 00:00 with 96 periods
print(newData.tail(5)) # Last Day Ending at 23:45 with 96 periods
print(len(newData))
newData = newData.dropna()
print(len(newData))

Total Days:  30.0
                       spots_available
verification_datetime                 
2016-10-03 00:00:00             4651.0
2016-10-03 00:15:00             4652.0
2016-10-03 00:30:00             4652.5
2016-10-03 00:45:00             4653.0
2016-10-03 01:00:00             4656.0
                       spots_available
verification_datetime                 
2016-11-01 22:45:00             5171.0
2016-11-01 23:00:00             5180.0
2016-11-01 23:15:00             5189.0
2016-11-01 23:30:00             5198.0
2016-11-01 23:45:00             5198.0
2880
2880


In [43]:
df = newData.to_numpy()
df = df.reshape(1,len(newData))
df = df.tolist()
print(len(df[0]))
print(df)

2880
[[4651.0, 4652.0, 4652.5, 4653.0, 4656.0, 4657.0, 4658.0, 4663.0, 4665.0, 4665.0, 4665.5, 4666.0, 4667.0, 4667.0, 4666.5, 4666.0, 4667.0, 4667.0, 4666.0, 4666.0, 4667.0, 4666.0, 4666.0, 4666.0, 4664.0, 4664.0, 4663.0, 4663.0, 4662.0, 4654.0, 4646.0, 4622.0, 4607.0, 4587.0, 4567.0, 4538.0, 4497.0, 4461.0, 4391.0, 4321.0, 4210.0, 4210.0, 3962.5, 3715.0, 3533.0, 3351.0, 3171.5, 2992.0, 2992.0, 2728.0, 2464.0, 2377.5, 2291.0, 2195.0, 2122.0, 2122.0, 2048.5, 1975.0, 1931.0, 1909.0, 1905.5, 1902.0, 1898.0, 1894.0, 1883.0, 1935.5, 1988.0, 2087.0, 2133.0, 2198.5, 2264.0, 2366.0, 2481.0, 2511.0, 2568.0, 2636.0, 2636.0, 2767.5, 2899.0, 3009.0, 3106.0, 3191.0, 3275.0, 3360.0, 3500.0, 3658.0, 3816.0, 3974.0, 4124.0, 4124.0, 4284.5, 4445.0, 4491.0, 4525.0, 4525.0, 4537.5, 4550.0, 4550.0, 4551.5, 4553.0, 4555.0, 4555.5, 4556.0, 4556.0, 4555.0, 4556.0, 4558.0, 4557.0, 4558.0, 4557.0, 4557.5, 4558.0, 4558.0, 4558.0, 4558.0, 4558.0, 4558.0, 4558.0, 4558.0, 4558.0, 4556.0, 4547.0, 4519.0, 4461.0, 4

In [9]:
# split a univariate sequence into samples
def split_sequence_numpy(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
        # gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [44]:
# choose a number of time steps
n_steps = 96
# split into samples
X, y = split_sequence_numpy(df[0], n_steps)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print("Done Steps")

Done Steps


In [11]:
#x_input = array(df[0][-96:])
#print(x_input)
#print(len(x_input))

[4011.  4011.  4015.5 4020.  4020.  4020.  4021.5 4023.  4024.  4025.
 4024.5 4024.  4024.  4024.  4023.  4023.  4024.  4024.5 4025.  4025.
 4025.  4025.  4025.  4025.  4022.  4011.  3979.  3910.  3831.  3683.
 3478.  3238.5 2999.  2667.5 2336.  2013.  1719.  1719.  1719.  1719.
 1719.  1719.  1719.  1719.  1719.  2005.5 2292.  2231.  2173.  2125.
 2077.  2035.  2014.  1966.  1916.  1852.  1798.  1816.  1834.  1839.
 1860.  1899.  1902.  1942.5 1983.  2037.  2087.  2220.  2310.  2476.
 2653.  2832.  2961.  2961.  3218.5 3476.  3614.  3692.  3840.  3988.
 4161.  4334.  4471.  4608.  4720.  4844.  4991.  5128.  5265.  5409.
 5500.  5500.  5499.  5497.  5498.  5499. ]
96


In [74]:
print("Fitting Model")
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))

opt = SGD(lr=0.01, momentum=0.9, clipvalue=1)

model.compile(optimizer=opt, loss='mse')
# fit model
model.fit(X, y, epochs=5, verbose=1)
print("Fit Model Done!")

Fitting Model
Epoch 1/5
2784/2784 [==============================] - 6s 2ms/step - loss: 129015175198902251742984518238208.0000
Epoch 2/5
2784/2784 [==============================] - 4s 1ms/step - loss: 72339001461296842748714811392.0000
Epoch 3/5
2784/2784 [==============================] - 4s 1ms/step - loss: 416998650357773238164272556539904.0000
Epoch 4/5
2784/2784 [==============================] - 4s 1ms/step - loss: 2564471.5661
Epoch 5/5
2784/2784 [==============================] - 4s 1ms/step - loss: 2723702.2256
Fit Model Done!


In [75]:
initial = 96*12
predict_day = initial+96

#x_input = array(df[0][initial:predict_day])
x_input = data[initial:predict_day].to_numpy()
x_input = array(x_input)

#print(x_input)
print(x_input[-3:])
print("\n")

# demonstrate prediction

x_input = x_input.reshape((1, n_steps, n_features))
#print(x_input)
print("Reshape Model Done")

print("Predicting...")
yhat = model.predict(x_input, verbose=1)
print("\nPredict: ",yhat)
print("\nReal: ",data[predict_day:predict_day+1])

[[4438.]
 [4477.]
 [4498.]]


Reshape Model Done
Predicting...
1/1 [==============================] - 1s 1s/step

Predict:  [[2889.3376]]

Real:                         spots_available
verification_datetime                 
2016-10-16                      4519.0
